# Setup Multiple Files

Import libraries

In [ ]:
import os
from os import listdir
from os.path import isfile, join
import shutil
import zipfile
import pickle
import re
import csv
import numpy as np
from scipy.signal import butter, lfilter, freqz
import scipy.signal as sg
import pandas as pd
import xlrd

import matplotlib.pyplot as plt
%matplotlib inline


import scipy as sp
from scipy import signal

Open joint angle data

In [ ]:
def open_angle_data(a_data):
    with open(a_data, 'r') as file:
        try:
            data = file.read()
            file.close()
        except UnicodeDecodeError:
            file.close()
    data = data.splitlines()
    data = data[4:len(data)-1] #Text information on first four rows and END on last row

    data_dim = len(data[0].split()[1:])

    traj_data = np.ndarray((len(data),data_dim))
    for i, row in enumerate(data):
        traj_data[i] = np.array([float(n)*np.pi/180 for n in row.split()[1:]])
    
    return traj_data

Open voltage and current data

In [ ]:
def open_power_data(p_data):
    f = open(p_data)

    data = []
    for line in f:
        data_line = line.rstrip().split('\t')
        data.append(data_line)

    init=True
    for i, file in enumerate(data[9:]):
        file = file[0].replace(',','.')
        file = file.replace(' ','')
        if init:
            Volt_Amp_data = np.ndarray((len(data),len(file.split(';'))-1))
            init=False
        Volt_Amp_data[i] = np.array([float(n) for n in file.split(';')[:len(file.split(';'))-1]])
   
    return Volt_Amp_data

Open temperature data

In [ ]:
def open_temp_data(temp_data_path):
    temp = pd.read_excel(temp_data_path)
    temp = temp.values
    temp_data = temp[0][3:]
    
    return temp_data

Interpolate temerature data

In [ ]:
def interpolate_temp(temp_data,traj_data_synced):
    n_timesteps = len(traj_data_synced)
    num_joints=6
    temp_lin_int = np.ndarray((n_timesteps,num_joints))
    for i in range(num_joints):
        temp_lin_int[0:,i] = np.linspace(temp_data[0+i],temp_data[6+i],num=n_timesteps)
    return temp_lin_int

Calculate momentaneous power form voltage/current measurements

In [ ]:
def calc_moment_power(Volt_Amp_data):
    n_phases   = 3   #Number of phases in measured data
    power_data = np.ndarray((len(Volt_Amp_data),1))
    amp_data   = np.ndarray((len(Volt_Amp_data),n_phases))
    volt_data  = np.ndarray((len(Volt_Amp_data),n_phases))

    for i, sample in enumerate(Volt_Amp_data):
        volt_data[i]  = np.array([sample[1], sample[2], sample[3]])
        amp_data[i]   = np.array([sample[4], sample[5], sample[6]])
        power_data[i] = np.abs(sample[1]*sample[4]) \
            + np.abs(sample[2]*sample[5]) \
            + np.abs(sample[3]*sample[6]) 
    
    return power_data, volt_data

Calculate moving average of momentaneous power. Moving average is calculated for one period at a time so that the current spikes behaves similarly in every calculated average. Then the average is upsampled with a factor of 5 (from 50Hz to 250 Hz) because this is an even factor of 3 times the sample frequency of the angle measurements which will be upsampled to to the same frequency later.

In [ ]:
def Downsample_powerdata(volt_data, power_data):
    period    = 200
    power_data_250 = np.ndarray((len(range(period,len(volt_data),period))*5,1))
    start = np.argmin(volt_data[0:200])

    i=0
    for j in range(start,len(power_data),period):
        power_data_250[i:i+5] = np.mean(power_data[j:j+period])
        i += 5

    #Linear interpolation instead of constant value upsampling
    i=0
    lin_in=np.ndarray((5,1))
    for j in range(0,len(power_data_250),5):
        a = power_data_250[i]
        if i+6 > len(power_data_250):
            break
        b = power_data_250[i+6]
        lin_in = np.linspace(a,b,num=5).reshape(5,1)
        power_data_250[i:i+5] = lin_in
        i+=5
    
    return power_data_250

Calculate velocity, acceleration and pseudopower

In [ ]:
def calc_vel_acc_psi(traj_data):
    #Time vector for angle measurements (fixed in KUKA system)
    delta_t = 0.012
    t = np.ndarray((len(traj_data),1))
    t[0] = 0
    for i in range(1,len(traj_data)):
        t[i] = t[i-1] + delta_t

    #Velocity 
    vel = np.ndarray(np.shape(traj_data))
    for i in range(len(traj_data)-1):
        vel[i] = np.divide( traj_data[i+1] - traj_data[i], delta_t)
    vel[-1] = vel[-2]

    #Acceleration
    acc = np.ndarray(np.shape(traj_data))
    for i in range(len(traj_data)-1):
        acc[i] = np.divide( vel[i+1] - vel[i], delta_t)
    acc[-1] = acc[-2]

    #Pseudopower
    psi = np.ndarray((np.shape(traj_data)[0],1))
    for i in range(len(traj_data)):
        psi[i] = np.sum(np.abs(vel[i]*acc[i]))
        
    return vel, acc, psi

Upsample angle measurement data to 250 Hz using linear interpolation.

In [ ]:
def Upsample_ang_vel_acc_psi(traj_data,vel,acc,psi):
    US = 3 #Upsample factor
    traj_data_us = np.ndarray((len(traj_data)*US,6))
    vel_us = np.ndarray(np.shape(traj_data_us))
    acc_us = np.ndarray(np.shape(traj_data_us))
    psi_us = np.ndarray((len(traj_data)*US,1))
    k=0
    for i, angles in enumerate(traj_data[0:-1]):
        for j in range(len(angles)):
            a = traj_data[i][j]
            b = traj_data[i+1][j]
            lin_in = np.linspace(a,b,num=US)
            traj_data_us[k:k+US,j] = lin_in

            a = vel[i][j]
            b = vel[i+1][j]
            lin_in = np.linspace(a,b,num=US)
            vel_us[k:k+US,j] = lin_in

            a = acc[i][j]
            b = acc[i+1][j]
            lin_in = np.linspace(a,b,num=US)
            acc_us[k:k+US,j] = lin_in

        a = psi[i]
        b = psi[i+1]
        lin_in = np.linspace(a,b,num=US).reshape(US,1)
        psi_us[k:k+US] = lin_in

        k+=US
        
    return traj_data_us, vel_us, acc_us, psi_us

Find start of power- and angle measurements

In [ ]:
def find_starts(power_data_250, traj_data_us, acc_us):
    Start_power_indicator = 300
    for i in range(len(power_data_250)):
        if power_data_250[i+10]-power_data_250[i]>Start_power_indicator:
            starting_point_power = i
            break
    #print('Start at power sample:',starting_point_power-1,' corresponding to time:',power_time_ds[starting_point_power-1],'sec')

    Start_trajectory_indicator = 0.5
    for i in range(len(traj_data_us)):
        if np.max(np.abs(acc_us[i+10]))>Start_trajectory_indicator:
            starting_point_traj = i
            break
    #print('Start at trajectory sample:',starting_point_traj,' corresponding to time:',t[starting_point_traj],'sec')
    return starting_point_power, starting_point_traj

In [ ]:
def align_data(traj_data_us,vel_us,acc_us,psi_us,starting_point_traj,power_data_250,starting_point_power):
    #Align the data
    traj_data_synced = traj_data_us[starting_point_traj:-1]
    vel_synced = vel_us[starting_point_traj:-1]
    acc_synced = acc_us[starting_point_traj:-1]
    psi_synced = psi_us[starting_point_traj:-1]
    power_data_synced = power_data_250[starting_point_power-1:starting_point_power-1+len(traj_data_synced)]
    return traj_data_synced, vel_synced, acc_synced, psi_synced, power_data_synced

Augment the input vector and save

In [ ]:
def create_aug_input(traj_data_synced, vel_synced, acc_synced, psi_synced, power_data_synced, temp_data):
    #Create the augmented input vector containing [angle1,..,angle6, vel1,..,vel6, acc1,..,acc6, psi] for all timesteps.
    aug_feature_dim = 25
    aug_input = np.ndarray((len(traj_data_synced),aug_feature_dim))

    for i in range(len(traj_data_synced)):
        aug_input[i] = np.concatenate((traj_data_synced[i], vel_synced[i], acc_synced[i], psi_synced[i], temp_data[i]), axis=None)

    return aug_input

Normalize input data within axis range

In [ ]:
def normalize_inputs(traj_data_synced, vel_synced, acc_synced, psi_synced, power_data_synced, temp_data):
    feature_dim = len(traj_data_synced[0])*4 + 1

    aug_input_norm = np.ndarray((len(traj_data_synced),feature_dim))
    traj_data_norm = np.ndarray(np.shape(traj_data_synced))
    vel_norm = np.ndarray(np.shape(vel_synced))
    acc_norm = np.ndarray(np.shape(acc_synced))
    psi_norm = np.ndarray(np.shape(psi_synced))
    temp_norm = np.ndarray(np.shape(temp_data))
        
    A1_constraint = [np.deg2rad(-185), np.deg2rad(185)]
    A2_constraint = [np.deg2rad(-135), np.deg2rad(35)]
    A3_constraint = [np.deg2rad(-120), np.deg2rad(158)]
    A4_constraint = [np.deg2rad(-350), np.deg2rad(350)]
    A5_constraint = [np.deg2rad(-119), np.deg2rad(119)]
    A6_constraint = [np.deg2rad(-350), np.deg2rad(350)]

    vel_max = 4.159647569601188
    vel_min = -3.9450433392170803
    acc_max = 27.338764681179683
    acc_min = -28.319179147808107
    psi_max = 126.43383111466377
    psi_min = 0.0
    temp_max = 340
    temp_min = 285

    constraints_min = [A1_constraint[0],A2_constraint[0],A3_constraint[0],A4_constraint[0],A5_constraint[0],A6_constraint[0]]
    constraints_diff = [float(np.diff(A1_constraint)), float(np.diff(A2_constraint)),float(np.diff(A3_constraint)),float(np.diff(A4_constraint)),float(np.diff(A5_constraint)),float(np.diff(A6_constraint))]

    vel_diff = vel_max - vel_min
    acc_diff = acc_max - acc_min
    psi_diff = psi_max - psi_min
    temp_diff = temp_max - temp_min

    for i in range(len(traj_data_synced)):
        traj_data_norm[i] = 2*np.divide((traj_data_synced[i] - constraints_min),constraints_diff) - 1
        vel_norm[i] = 2*(vel_synced[i] - vel_min)/vel_diff - 1
        acc_norm[i] = 2*(acc_synced[i] - acc_min)/acc_diff - 1
        psi_norm[i] = psi_synced[i]/psi_diff    
        temp_norm[i] = 2*(temp_data[i] - temp_min)/temp_diff - 1
        aug_input_norm[i] = np.concatenate((traj_data_norm[i], vel_norm[i], acc_norm[i], psi_norm[i], temp_norm[i]), axis=None)
        
    return aug_input_norm


In [ ]:
def save_data(aug_input,name_aug,aug_input_norm,name_norm,power_data_synced,name_power):
    with open(name_aug, 'wb') as file:
        pickle.dump(aug_input, file)
        file.close()

    with open(name_norm, 'wb') as file:
        pickle.dump(aug_input_norm, file)
        file.close()

    with open(name_power, 'wb') as file:
        pickle.dump(power_data_synced, file)
        file.close()

Loop through all data files 

In [ ]:
def process_all_data():
    
    angle_datapath = 'Collected data/AxisData/'
    power_datapath = 'Collected data/PowerData/'
    temp_datapath = 'Collected data/TempData/'
    
    angle_files = [f for f in listdir(angle_datapath) if isfile(join(angle_datapath, f))]
    power_files = [f for f in listdir(power_datapath) if isfile(join(power_datapath, f))]
    temp_files = [f for f in listdir(temp_datapath) if isfile(join(temp_datapath, f))]
    
    
    for i in range(len(power_files)):
        print('Processing angle file: ' + angle_files[i])
        print('Processing power file: ' + power_files[i])
        print('Processing temperature file: ' + temp_files[i])
        print('')
        a_data = angle_datapath + angle_files[i] #The angle data
        p_data = power_datapath + power_files[i] #The power data
        temp_data_path = temp_datapath + temp_files[i] #Temperature data
        
        traj_data = open_angle_data(a_data)
        vel, acc, psi = calc_vel_acc_psi(traj_data)
        traj_data_us, vel_us, acc_us, psi_us = Upsample_ang_vel_acc_psi(traj_data,vel,acc,psi)
        
        Volt_Amp_data = open_power_data(p_data)
        power_data, volt_data = calc_moment_power(Volt_Amp_data)
        power_data_250 = Downsample_powerdata(volt_data, power_data)
        
        starting_point_power, starting_point_traj = find_starts(power_data_250, traj_data_us, acc_us)
        traj_data_synced, vel_synced, acc_synced, psi_synced, power_data_synced = align_data(traj_data_us,vel_us,acc_us,psi_us,starting_point_traj,power_data_250,starting_point_power)
        temp_data = open_temp_data(temp_data_path)
        temp_lin_int = interpolate_temp(temp_data,traj_data_synced)
        
        aug_input = create_aug_input(traj_data_synced, vel_synced, acc_synced, psi_synced, power_data_synced,temp_lin_int)
        aug_input_norm = normalize_inputs(traj_data_synced, vel_synced, acc_synced, psi_synced, power_data_synced,temp_lin_int)
        
        if not os.path.isdir('Pickled_data'):
            os.mkdir('Pickled_data')
        
        pickled_datapath = 'Pickled_data'
        path_aug = pickled_datapath + '/' + angle_files[i].replace('.txt','') + '_auginput.pickle'
        path_norm = pickled_datapath + '/'+angle_files[i].replace('.txt','') + '_augnorminput.pickle'
        path_power = pickled_datapath + '/'+angle_files[i].replace('.txt','') + '_poweroutput.pickle'
        save_data(aug_input,path_aug,aug_input_norm,path_norm,power_data_synced,path_power)
    

# Now everything is defined, time to process all data files!

In [ ]:
process_all_data()